## Prompt Templating
- How to prepare prompt template using ibm sdk.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
my_api_key = os.getenv("GENAI_KEY", None)
my_api_endpoint = os.getenv("GENAI_API", None)

### How to design prompt template using `ibm_generative_ai`
- How templates renders with `PromptPattern`

In [3]:
from genai.prompt_pattern import PromptPattern
from genai.credentials import Credentials

## 1. Template
```yaml
"""
{{instruction}}

{{#list}}

Country: {{country}}
Airport: {{airport}}
Capital: {{capital}}

{{/list}}
Country: {{input}}
"""
```
## Output:
```yaml
#Examples
Country: Canada
Airport: YOW
Capital: Ottawa


Country: Germany
Airport: BER
Capital: Berlin


Country: USA
Airport: DCA
Capital: Washington

#input
Country: Bangladesh
```

In [4]:
creds = Credentials(api_key=my_api_key, api_endpoint=my_api_endpoint)

_template = """
{{instruction}}

{{#list}}

Country: {{country}}
Airport: {{airport}}
Capital: {{capital}}

{{/list}}
Country: {{input}}
"""

print("\n------------- Mustaches Prompt Template -------------\n")
pt = PromptPattern.from_watsonx(credentials=creds, name="list-qa", template=_template)
print(f"\nPrompt: {pt}")


print("\n------------- Rendered Prompt -------------\n")
inputs = ["Spain", "Finland", "Iraq", "India", "Bangladesh"]
data = {
    "list": [
        {"country": "Canada", "capital": "Ottawa", "airport": "YOW"},
        {"country": "Germany", "capital": "Berlin", "airport": "BER"},
        {"country": "USA", "capital": "Washington", "airport": "DCA"},
    ]
}

rendered_prompts = pt.render(inputs=inputs, data=data)
for pt in rendered_prompts:
    print(pt)
    print("- - - - ")


------------- Mustaches Prompt Template -------------


Prompt: 
{{instruction}}

{{#list}}

Country: {{country}}
Airport: {{airport}}
Capital: {{capital}}

{{/list}}
Country: {{input}}


------------- Rendered Prompt -------------





Country: Canada
Airport: YOW
Capital: Ottawa


Country: Germany
Airport: BER
Capital: Berlin


Country: USA
Airport: DCA
Capital: Washington

Country: Spain

- - - - 




Country: Canada
Airport: YOW
Capital: Ottawa


Country: Germany
Airport: BER
Capital: Berlin


Country: USA
Airport: DCA
Capital: Washington

Country: Finland

- - - - 




Country: Canada
Airport: YOW
Capital: Ottawa


Country: Germany
Airport: BER
Capital: Berlin


Country: USA
Airport: DCA
Capital: Washington

Country: Iraq

- - - - 




Country: Canada
Airport: YOW
Capital: Ottawa


Country: Germany
Airport: BER
Capital: Berlin


Country: USA
Airport: DCA
Capital: Washington

Country: India

- - - - 




Country: Canada
Airport: YOW
Capital: Ottawa


Country: Germany
Airport: BER


## 2. Prepare Prompts templates for CSV files

In [23]:
import requests
import pandas as pd
import os
import pathlib

In [25]:
def get_dataset(datafile):
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

    if os.path.exists(datafile):
        print("Data already downloaded. Skipping the download.")
        return
    with requests.get(url) as response:
        print("Saving data to {}".format(datafile))
        # Add data
        with open(datafile, "ab") as f:
            f.write(response.content)


datafile = os.path.join(pathlib.Path('__file__').parent.absolute(), "adult.data.csv")
get_dataset(datafile)

Saving data to C:\Users\003EPO744\Desktop\LearningTechs\GenerativeAI\GenAI With IBM BAM\adult.data.csv


In [26]:
data_df = pd.read_csv('adult.data.csv',header=None)
data_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [27]:
template = """
    The age of the individual is {{0}}.
    The workclass of the individual is {{1}}.
    The estimate of the number of individuals in the population with the same demographics as the individual was {{2}}.
    The numeric form of the highest education level they achieved was {{4}}.
    The marital status of the individual was {{5}}.
    The occupation of the individual was {{6}}.
    The relationship status of the individual is {{7}}.
    The race of the individual was {{8}}.
    The sex of the individual is {{9}}.
    The capital gain they had in the previous year was {{10}}.
    The capital loss they had in the previous year was {{11}}.
    The number of hours worked per week was {{12}}.
    The native country of the individual was {{13}}.
"""

In [28]:
prompt = PromptPattern.from_str(template)
list_of_prompts = prompt.sub_all_from_csv(csv_path=datafile, col_to_var="infer", headers=False)

In [30]:
list_of_prompts[:3]

[
     The age of the individual is 39.
     The workclass of the individual is  State-gov.
     The estimate of the number of individuals in the population with the same demographics as the individual was  77516.
     The numeric form of the highest education level they achieved was  13.
     The marital status of the individual was  Never-married.
     The occupation of the individual was  Adm-clerical.
     The relationship status of the individual is  Not-in-family.
     The race of the individual was  White.
     The sex of the individual is  Male.
     The capital gain they had in the previous year was  2174.
     The capital loss they had in the previous year was  0.
     The number of hours worked per week was  40.
     The native country of the individual was  United-States.,
 
     The age of the individual is 50.
     The workclass of the individual is  Self-emp-not-inc.
     The estimate of the number of individuals in the population with the same demographics as the indivi